# 2019/09/12 ver.1.0
## 一、文件内容
从文件引入数据<br />
从文件引入模型<br />
 1：支持向量回归机<br />
 2：随机森林回归<br />
## 二、数据集
处理后的原训练集 特征数据：dg_pca_train<br />
处理后的原训练集合 得分数据：score_train<br />
处理后的测试集合：dg_pca_test<br />
测试集序号：id_test<br />

在dg_pca_train中继续划分数据<br />
X_train：0.8比例的原训练集特征 用作训练和验证<br />
X_test：0.2比例的原训练集特征 用作测试（模型选择）<br />
y_train：0.8比例的原训练集分数 对应X_train<br />
y_test：0.2比例的原训练集分数 对应X_test<br />
## 三、单模型信息


文件名|变量名|模型名|超参数设置|训练集上预测值|最终结果|是否导入
:--------------:|:---------:|:------------------:|:-----------------:|:-------------------------:|:----------------:|:----:|
SVR.pickle|svc|支持向量回归|C=10 gamma=0.001|0.5221925320557665|0.5311109175460105|
svc_g.pickle|svc_g|支持向量回归|C=100 gamma=0.001|0.5234655148127494|0.5335009474194661|
svc_best.pickle|svc_best|支持向量回归|C=10,gamma=0.0001|0.556846220888782|0.5533334949038184|√
svcRF.pickle|svcRF|随机森林回归|n_estimators=300|0.3619639542577069|NA|
svcRF_best.pickle|svcRF_best|随机森林回归|n_estimators=500|0.36527243324366043|NA|√
xgbr.pickle|xgbr|Xgboost回归|n_estimators=800|0.4505720493043171|NA|
xgbr_1100.pickle|xgbr_1100|Xgboost回归|n_estimators=1100|0.4600838333957028|NA|√

## 四、交叉验证模型信息

文件名|变量名|模型名|参数范围
:-------:|:-----:|:-----:|:-----:
cv_svc.pickle|NA|支持向量回归|"C": [1e0, 1e1, 1e2, 1e3],<br>"gamma": np.logspace(-4, 0, 5)
cv_svcRF.pickle|NA|随机森林回归|"n_estimators":[20,50,100,150,200,260,300,500]

### 以下为预设导入部分

In [1]:
#预设导入
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import joblib

import warnings
warnings.filterwarnings("ignore")

In [105]:
#机器学习导入
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score

In [3]:
#读取本地数据集
dg_pca_train=pd.read_csv('dg_pca_train.csv').values
dg_pca_test=pd.read_csv('dg_pca_test.csv').values

id_test=pd.read_csv('id_test.csv').values
id_train=pd.read_csv('id_train.csv').values
score_train=pd.read_csv('score_train.csv').values
id_train=id_train.flatten()
id_test=id_test.flatten()#ID二维数组转一维

In [80]:
#载入模型
#svc = joblib.load('SVR.pickle')
#svcRF = joblib.load('svcRF.pickle')
svc_best = joblib.load('svc_best.pickle')
svcRF_best = joblib.load('svcRF_best.pickle')
xgbr_1100=joblib.load('xgbr_1100.pickle')

[14:30:09] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [93]:
# #如有需要可以载入的参数选择模型
# cv_svc = joblib.load('cv_svc.pickle')
# cv_svcRF = joblib.load('cv_svcRF.pickle')
# #cv_xgbr = joblib.load('cv_xgbr.pickle')

In [4]:
#划分训练数据集合
#80%的训练集合 20%的模型测试集合 随机种子24
X_train, X_test, y_train, y_test = train_test_split(dg_pca_train, score_train, test_size=0.2, random_state=24)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((10984, 941), (10984, 1), (2747, 941), (2747, 1))

### 以下进行模型进一步的分析工作

In [66]:
#svm回归模型 C=10,gamma=0.0001
svc_best=svm.SVR(kernel='rbf',C=10,gamma=0.0001)
svc_best.fit(X_train,y_train)

print(svc_best.score(X_test,y_test))

0.556846220888782


In [76]:
#保存模型
joblib.dump(clf, 'cv_svc.pickle')#C=10,gamma=0.0001
#载入模型
svc_best = joblib.load('svc_best.pickle')


['cv_svc.pickle']

In [71]:
svc_best = joblib.load('svc_best.pickle')

In [72]:
print(svc_best.score(X_test,y_test))

0.556846220888782


In [78]:

xgbr=joblib.load('xgbr.pickle')

[13:09:14] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [79]:
print(xgbr.score(X_test,y_test))

0.4505720493043171


In [ ]:
cv_params = [{'n_estimators': [700,800,900,1000,1100]}]

scores_GBM=['r2']

for score in scores_GBM:
    print(score)
    model = xgb.XGBRegressor()
    optimized_GBM=GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=3, verbose=1, n_jobs=-1)
    optimized_GBM.fit(X_train,y_train)
    
    print(optimized_GBM.best_estimator_)
    
    means_GBM = optimized_GBM.cv_results_['mean_test_score']
    stds_GBM=optimized_GBM.cv_results_['std_test_score']
    params_GBM = optimized_GBM.cv_results_['params']
    
    for (param_GBM,mean_score_GBM,std_score_GBM) in zip(params_GBM,means_GBM,stds_GBM):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM,std_score_GBM,param_GBM))

In [89]:
print(svc_best.score(X_test,y_test))

y_pred = svc_best.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.556846220888782
0.556846220888782


In [90]:
print(svcRF_best.score(X_test,y_test))

y_pred = svcRF_best.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.36527243324366043
0.36527243324366043


In [91]:
print(xgbr_1100.score(X_test,y_test))

y_pred = xgbr_1100.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.4600838333957028
0.4600838333957028


In [99]:
clfs = [
        RandomForestRegressor(),
        svm.SVR(),
        XGBRegressor()
    ]
for j, clf in enumerate(clfs):
        print('Training classifier [%s]' % (j))

Training classifier [0]
Training classifier [1]
Training classifier [2]


In [ ]:
from sklearn.model_selection import StratifiedKFold

# '''创建训练的数据集'''
#data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)
 
# '''模型融合中使用到的各个单模型'''
clfs = [RandomForestRegressor(),
        svm.SVR(),
        XGBRegressor()]
 
#'''切分一部分数据作为测试集'''
X=dg_pca_train
X_predict=dg_pca_test
y=score_train
#y_predict = 


dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))

#'''5折stacking'''
n_folds = 5
skf = list(StratifiedKFold(y, n_folds,True,random_state=50))

for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    # print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        # print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print("val auc Score: %f" % roc_auc_score(y_predict, dataset_blend_test[:, j]))
    
# # clf = LogisticRegression()
# clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
# clf.fit(dataset_blend_train, y)
# y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
 
# print("Linear stretch of predictions to [0,1]")
# y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
# print("blend result")
# print("val auc Score: %f" % (roc_auc_score(y_predict, y_submission)))

In [ ]:
def stacking_first(train, train_y, test):
    savepath = './stack_op{}_dt{}_tfidf{}/'.format(args.option, args.data_type, args.tfidf)
    os.makedirs(savepath, exist_ok=True)

    count_kflod = 0
    num_folds = 6
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=10)
    # 测试集上的预测结果
    predict = np.zeros((test.shape[0], config.n_class))
    # k折交叉验证集的预测结果
    oof_predict = np.zeros((train.shape[0], config.n_class))
    scores = []
    f1s = []

    for train_index, test_index in kf.split(train):
        # 训练集划分为6折，每一折都要走一遍。那么第一个是5份的训练集索引，第二个是1份的测试集，此处为验证集是索引

        kfold_X_train = {}
        kfold_X_valid = {}

        # 取数据的标签
        y_train, y_test = train_y[train_index], train_y[test_index]
        # 取数据
        kfold_X_train, kfold_X_valid = train[train_index], train[test_index]

        # 模型的前缀
        model_prefix = savepath + 'DNN' + str(count_kflod)
        if not os.path.exists(model_prefix):
            os.mkdir(model_prefix)

        M = 4  # number of snapshots
        alpha_zero = 1e-3  # initial learning rate
        snap_epoch = 16
        snapshot = SnapshotCallbackBuilder(snap_epoch, M, alpha_zero)

        # 使用训练集的size设定维度，fit一个模型出来
        res_model = get_model(train)
        res_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # res_model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCH, verbose=1,  class_weight=class_weight)
        res_model.fit(kfold_X_train, y_train, batch_size=BATCH_SIZE, epochs=snap_epoch, verbose=1,
                      validation_data=(kfold_X_valid, y_test),
                      callbacks=snapshot.get_callbacks(model_save_place=model_prefix))

        # 找到这个目录下所有已经训练好的深度学习模型，通过".h5"
        evaluations = []
        for i in os.listdir(model_prefix):
            if '.h5' in i:
                evaluations.append(i)

        # 给测试集和当前的验证集开辟空间，就是当前折的数据预测结果构建出这么多的数据集[数据个数，类别]
        preds1 = np.zeros((test.shape[0], config.n_class))
        preds2 = np.zeros((len(kfold_X_valid), config.n_class))
        # 遍历每一个模型，用他们分别预测当前折数的验证集和测试集，N个模型的结果求平均
        for run, i in enumerate(evaluations):
            res_model.load_weights(os.path.join(model_prefix, i))
            preds1 += res_model.predict(test, verbose=1) / len(evaluations)
            preds2 += res_model.predict(kfold_X_valid, batch_size=128) / len(evaluations)

        # 测试集上预测结果的加权平均
        predict += preds1 / num_folds
        # 每一折的预测结果放到对应折上的测试集中，用来最后构建训练集
        oof_predict[test_index] = preds2

        # 计算精度和F1
        accuracy = mb.cal_acc(oof_predict[test_index], np.argmax(y_test, axis=1))
        f1 = mb.cal_f_alpha(oof_predict[test_index], np.argmax(y_test, axis=1), n_out=config.n_class)
        print('the kflod cv is : ', str(accuracy))
        print('the kflod f1 is : ', str(f1))
        count_kflod += 1

        # 模型融合的预测结果，存起来，用以以后求平均值
        scores.append(accuracy)
        f1s.append(f1)
    # 指标均值，最为最后的预测结果
    print('total scores is ', np.mean(scores))
    print('total f1 is ', np.mean(f1s))
    return predict

In [117]:
#Kfold 用法 不会因为刷新而变化 返回的是index
from sklearn.model_selection import KFold
kf = KFold(5, True,50)

for train_index, test_index in kf.split(X_train):
    kfold_X={}
    kfold_Y={}
    kfold_X,kfold_Y=X_train[train_index],X_test[test_index]
#    print('训练集:{}'.format(train_index))
#    print('测试集:{}'.format(test_index))
#    print(kfold_X)
#    print(kfold_Y)
#    print(Y[train_index])
#    print(Y[test_index])
#    print(type(X_train[test_index]))
    
    #model.fit()


IndexError: index 2755 is out of bounds for axis 0 with size 2747

In [ ]:
X=[1:30]

In [103]:
#在独立测试集上预测数据
svc_best_predict=svc_best.predict(dg_pca_test)
print(svc_best_predict)
np.shape(svc_best_predict)

KeyboardInterrupt: 

In [120]:
clfs = [RandomForestRegressor(n_estimators=500),
        svm.SVR(kernel='rbf',C=10,gamma=0.0001),
        XGBRegressor()]
n_folds = 5
kf = KFold(n_folds,True,50)
skf=list(kf.split(y_train))


for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_test.shape[0], len(skf)))#存目前这个模型上的测试集结果(之后求平均)
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        print("Fold", i)
        print(len(train),len(test))
        #X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        #clf.fit(X_train, y_train)
        #y_submission =y_pred#1fold的预测结果
        y_submission=np.arange(1,2198)
        dataset_blend_train[test, j] = y_submission
        #y_submission.reshape(len(y_submission),1)#一维数组转二维 说实话不知道需不需要 还是应该再研究一下
        #
        #dataset_blend_train[test, j] = #y_submission#在模型顺序对应的j位置 存1fold的预测结果
        #dataset_blend_test_j[:, i] = clf.predict(X_predict)#存该模型该折下的测试集预测结果
        
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    #dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)#测试集结果按行取平均后储存
    
    
    #print("val auc Score: %f" % r2_score(y_predict, dataset_blend_test[:, j]))
    
    #保存模型
    #fl_name="stacking_{0}".format(j)
    #joblib.dump(clf, fl_name)


0 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Fold 0
8787 2197
Fold 1
8787 2197
Fold 2
8787 2197
Fold 3
8787 2197
Fold 4
8788 2196
1 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Fold 0
8787 2197
Fold 1
8787 2197
Fold 2
8787 2197
Fold 3
8787 2197
Fold 4
8788 2196
2 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
         

In [130]:
dataset_blend_train = np.zeros((X_train.shape[0], len(clfs)))
y_submission=np.arange(1,2198)
y_submission.reshape(len(y_submission),1)
j=np.arange(3,2200)
dataset_blend_train[j,0] = y_submission

In [132]:
dataset_blend_train[2009,0]

2007.0

In [116]:
skf = KFold(n_folds,True,50)


AttributeError: 'KFold' object has no attribute 'shape'